In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import os

plt.style.use('seaborn-v0_8-whitegrid')

# 创建中文字体设置函数
def get_chinese_font():
    """获取中文字体对象"""
    font_path = 'C:/Windows/Fonts/simhei.ttf'
    if os.path.exists(font_path):
        return fm.FontProperties(fname=font_path)
    else:
        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False
        return None


# 加载数据
df = pd.read_csv('processed_dengue_data.csv')

# 转换日期列回为日期类型（CSV读取会将其变为字符串）
df['Date'] = pd.to_datetime(df['Date'])

# 计算全国级别的统计信息
total_cases = df['Dengue_Cases'].sum()
total_deaths = df['Dengue_Deaths'].sum()
avg_case_fatality = (total_deaths / total_cases) * 100

print(f"五年间总登革热病例数: {total_cases:,}")
print(f"五年间总死亡人数: {total_deaths:,}")
print(f"平均病死率: {avg_case_fatality:.2f}%")

# 按年份统计
yearly_stats = df.groupby('Year').agg({
    'Dengue_Cases': ['sum', 'mean', 'min', 'max'],
    'Dengue_Deaths': ['sum', 'mean', 'min', 'max']
})

print("\n按年度统计信息：")
print(yearly_stats)

# 按月份统计（季节性分析）
monthly_stats = df.groupby('Month').agg({
    'Dengue_Cases': ['sum', 'mean', 'min', 'max'],
    'Dengue_Deaths': ['sum', 'mean', 'min', 'max']
})

# 确保月份按正确顺序排列
monthly_stats = monthly_stats.reindex(
    ['January', 'February', 'March', 'April', 'May', 'June', 
     'July', 'August', 'September', 'October', 'November', 'December']
)

print("\n按月份统计信息（季节性）：")
print(monthly_stats)

# 按地区统计并找出前5个高发地区
region_stats = df.groupby('Region').agg({
    'Dengue_Cases': ['sum', 'mean', 'min', 'max'],
    'Dengue_Deaths': ['sum', 'mean', 'min', 'max']
})

# 按病例总数降序排序
region_stats = region_stats.sort_values(('Dengue_Cases', 'sum'), ascending=False)

print("\n按地区统计信息（前5个高发地区）：")
print(region_stats.head())

===== 开始下午的描述性统计分析与可视化 =====
五年间总登革热病例数: 1,147,425
五年间总死亡人数: 16,844
平均病死率: 1.47%

按年度统计信息：
     Dengue_Cases                          Dengue_Deaths                     
              sum         mean  min    max           sum       mean min   max
Year                                                                         
2016       209544  1027.176471   54   7155          8127  39.838235   0  1651
2017       154155   755.661765   17   4764          4563  22.367647   0   668
2018       250783  1229.328431   77   5581          1226   6.009804   0    44
2019       441902  2166.186275  163  21658          1733   8.495098   0   121
2020        91041   446.279412   10   3464          1195   5.857843   0   185

按月份统计信息（季节性）：
          Dengue_Cases                         Dengue_Deaths                 \
                   sum         mean min    max           sum       mean min   
Month                                                                         
January          84328   992.0941

In [18]:
# 1. 全国时间序列趋势图 
# 创建全国时间序列数据
national_timeseries = df.groupby(['Year', 'Month', 'MonthNum', 'Date']).agg({
    'Dengue_Cases': 'sum',
    'Dengue_Deaths': 'sum'
}).reset_index()

# 按日期排序
national_timeseries = national_timeseries.sort_values('Date')

# 确保字体设置在样式设置之后
plt.style.use('seaborn-v0_8-whitegrid')

# 创建中文字体对象（最可靠的方法）
import matplotlib.font_manager as fm
import os
font_path = 'C:/Windows/Fonts/simhei.ttf'
if os.path.exists(font_path):
    chinese_font = fm.FontProperties(fname=font_path)
    print("使用SimHei字体文件路径")
else:
    # 回退到rcParams方法
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    chinese_font = None
    print("使用rcParams设置SimHei字体")

plt.figure(figsize=(14, 7))
plt.plot(national_timeseries['Date'], national_timeseries['Dengue_Cases'], 
         marker='o', linestyle='-', markersize=4, linewidth=2)

# 使用字体对象显式设置所有文本元素
if chinese_font is not None:
    plt.title('菲律宾全国登革热病例时间序列趋势（2016-2020）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('日期', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('病例数', fontproperties=chinese_font, fontsize=12)
else:
    # 回退到默认方法
    plt.title('菲律宾全国登革热病例时间序列趋势（2016-2020）', fontsize=16)
    plt.xlabel('日期', fontsize=12)
    plt.ylabel('病例数', fontsize=12)

plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# 添加主要年份标记
plt.axvline(pd.to_datetime('2017-01-01'), color='gray', linestyle='--', alpha=0.7)
plt.axvline(pd.to_datetime('2018-01-01'), color='gray', linestyle='--', alpha=0.7)
plt.axvline(pd.to_datetime('2019-01-01'), color='gray', linestyle='--', alpha=0.7)
plt.axvline(pd.to_datetime('2020-01-01'), color='gray', linestyle='--', alpha=0.7)

# 保存图表
plt.savefig('全国登革热趋势.png', dpi=300)
print("保存图表：全国登革热趋势.png")
plt.close()


===== 开始数据可视化 =====
使用SimHei字体文件路径
保存图表：全国登革热趋势.png


In [19]:
# 2. 前5个高发地区比较图
# 找出前5个高发地区
top5_regions = region_stats.head(5).index.tolist()

# 设置样式并获取中文字体
plt.style.use('seaborn-v0_8-whitegrid')
chinese_font = get_chinese_font()

plt.figure(figsize=(14, 8))

# 为每个地区绘制一条线
for region in top5_regions:
    region_data = df[df['Region'] == region].groupby('Date')['Dengue_Cases'].sum().reset_index()
    region_data = region_data.sort_values('Date')
    plt.plot(region_data['Date'], region_data['Dengue_Cases'], label=region, linewidth=2)

# 使用中文字体设置标题和标签
if chinese_font is not None:
    plt.title('菲律宾前5个登革热高发地区比较（2016-2020）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('日期', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('病例数', fontproperties=chinese_font, fontsize=12)
    plt.legend(title='地区', prop=chinese_font)
else:
    plt.title('菲律宾前5个登革热高发地区比较（2016-2020）', fontsize=16)
    plt.xlabel('日期', fontsize=12)
    plt.ylabel('病例数', fontsize=12)
    plt.legend(title='地区')

plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# 保存图表
plt.savefig('前5个高发地区比较.png', dpi=300)
print("保存图表：前5个高发地区比较.png")
plt.close()

C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_16032\3778476911.py:31: UserWarning: Glyph 22320 (\N{CJK UNIFIED IDEOGRAPH-5730}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_16032\3778476911.py:31: UserWarning: Glyph 21306 (\N{CJK UNIFIED IDEOGRAPH-533A}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_16032\3778476911.py:34: UserWarning: Glyph 22320 (\N{CJK UNIFIED IDEOGRAPH-5730}) missing from font(s) Arial.
  plt.savefig('前5个高发地区比较.png', dpi=300)
C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_16032\3778476911.py:34: UserWarning: Glyph 21306 (\N{CJK UNIFIED IDEOGRAPH-533A}) missing from font(s) Arial.
  plt.savefig('前5个高发地区比较.png', dpi=300)


保存图表：前5个高发地区比较.png


In [20]:
# 3. 季节性模式图
# 按月份计算平均病例数（季节性分析）
monthly_avg = df.groupby(['MonthNum', 'Month'])['Dengue_Cases'].mean().reset_index()
monthly_avg = monthly_avg.sort_values('MonthNum')

# 设置样式并获取中文字体
plt.style.use('seaborn-v0_8-whitegrid')
chinese_font = get_chinese_font()

plt.figure(figsize=(12, 6))
ax = sns.barplot(x='Month', y='Dengue_Cases', data=monthly_avg, 
            order=['January', 'February', 'March', 'April', 'May', 'June', 
                   'July', 'August', 'September', 'October', 'November', 'December'])

# 使用中文字体设置标题和标签
if chinese_font is not None:
    plt.title('菲律宾登革热病例季节性模式（2016-2020平均值）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('月份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('平均病例数', fontproperties=chinese_font, fontsize=12)
    # 修改x轴刻度标签为中文月份名称
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    ax.set_xticklabels(month_names, fontproperties=chinese_font)
else:
    plt.title('菲律宾登革热病例季节性模式（2016-2020平均值）', fontsize=16)
    plt.xlabel('月份', fontsize=12)
    plt.ylabel('平均病例数', fontsize=12)

plt.xticks(rotation=45)
plt.grid(True, axis='y')
plt.tight_layout()

# 保存图表
plt.savefig('登革热季节性模式.png', dpi=300)
print("保存图表：登革热季节性模式.png")
plt.close()

C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_16032\608655814.py:23: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(month_names, fontproperties=chinese_font)


保存图表：登革热季节性模式.png


In [21]:
# 4. 按年度和月份创建热图，展示季节性模式
monthly_year_data = df.groupby(['Year', 'MonthNum']).agg({
    'Dengue_Cases': 'sum'
}).reset_index()

# 创建透视表用于热图
heatmap_data = monthly_year_data.pivot(index='MonthNum', columns='Year', values='Dengue_Cases')
heatmap_data.index = ['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月']

# 设置样式并获取中文字体
plt.style.use('seaborn-v0_8-whitegrid')
chinese_font = get_chinese_font()

plt.figure(figsize=(10, 8))

# 创建热图
sns.heatmap(heatmap_data, cmap='YlOrRd', annot=True, fmt='.0f', linewidths=0.5)

# 使用中文字体设置标题和标签
if chinese_font is not None:
    plt.title('登革热病例的年度和月度分布（热图）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('年份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('月份', fontproperties=chinese_font, fontsize=12)
else:
    plt.title('登革热病例的年度和月度分布（热图）', fontsize=16)
    plt.xlabel('年份', fontsize=12)
    plt.ylabel('月份', fontsize=12)

plt.tight_layout()

# 保存热图
plt.savefig('登革热年度月度热图.png', dpi=300)
print("保存图表：登革热年度月度热图.png")
plt.close()

c:\Users\nnnnghj\AppData\Local\Programs\Python\Python310\lib\site-packages\seaborn\utils.py:61: UserWarning: Glyph 26376 (\N{CJK UNIFIED IDEOGRAPH-6708}) missing from font(s) Arial.
  fig.canvas.draw()
C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_16032\2924099696.py:32: UserWarning: Glyph 26376 (\N{CJK UNIFIED IDEOGRAPH-6708}) missing from font(s) Arial.
  plt.savefig('登革热年度月度热图.png', dpi=300)


保存图表：登革热年度月度热图.png


In [22]:
# 5. 年度病例和死亡总数
yearly_totals = df.groupby('Year').agg({
    'Dengue_Cases': 'sum',
    'Dengue_Deaths': 'sum'
}).reset_index()

# 计算病死率
yearly_totals['Fatality_Rate'] = (yearly_totals['Dengue_Deaths'] / yearly_totals['Dengue_Cases']) * 100

# 设置样式并获取中文字体
plt.style.use('seaborn-v0_8-whitegrid')
chinese_font = get_chinese_font()

plt.figure(figsize=(12, 6))
ax = sns.barplot(x='Year', y='Dengue_Cases', data=yearly_totals)

# 在柱子上添加数值标签
for i, v in enumerate(yearly_totals['Dengue_Cases']):
    ax.text(i, v + 5000, f"{v:,.0f}", ha='center', fontsize=10)

# 使用中文字体设置标题和标签
if chinese_font is not None:
    plt.title('菲律宾登革热年度病例数比较（2016-2020）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('年份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('病例数', fontproperties=chinese_font, fontsize=12)
else:
    plt.title('菲律宾登革热年度病例数比较（2016-2020）', fontsize=16)
    plt.xlabel('年份', fontsize=12)
    plt.ylabel('病例数', fontsize=12)

plt.grid(True, axis='y')
plt.tight_layout()

# 保存图表
plt.savefig('登革热年度病例比较.png', dpi=300)
print("保存图表：登革热年度病例比较.png")
plt.close()

保存图表：登革热年度病例比较.png


In [23]:
# 6. 饼图展示前5个高发地区分布
top5_region_data = df[df['Region'].isin(top5_regions)].groupby('Region')['Dengue_Cases'].sum().reset_index()
other_regions = pd.DataFrame({
    'Region': ['其他地区'],
    'Dengue_Cases': [total_cases - top5_region_data['Dengue_Cases'].sum()]
})
pie_data = pd.concat([top5_region_data, other_regions])

# 设置样式并获取中文字体
plt.style.use('seaborn-v0_8-whitegrid')
chinese_font = get_chinese_font()

plt.figure(figsize=(10, 8))

# 创建饼图
wedges, texts, autotexts = plt.pie(pie_data['Dengue_Cases'], 
                                   labels=None,  # 暂时不设置标签
                                   autopct='%1.1f%%', 
                                   startangle=90, 
                                   shadow=True, 
                                   explode=[0.05]*len(pie_data))

# 使用中文字体设置标题和图例
if chinese_font is not None:
    plt.title('登革热病例区域分布（2016-2020）', fontproperties=chinese_font, fontsize=16)
    
    # 修改为中文标签和创建图例
    region_labels = list(pie_data['Region'])
    region_labels[-1] = '其他地区'  # 确保最后一个是"其他地区"
    
    plt.legend(wedges, region_labels, 
              title='地区', 
              loc='center left', 
              bbox_to_anchor=(1, 0, 0.5, 1),
              prop=chinese_font)
    
    # 设置百分比文本的字体
    for autotext in autotexts:
        autotext.set_fontproperties(chinese_font)
else:
    plt.title('登革热病例区域分布（2016-2020）', fontsize=16)
    
    # 使用默认字体设置
    region_labels = list(pie_data['Region'])
    region_labels[-1] = '其他地区'
    
    plt.legend(wedges, region_labels, 
              title='地区', 
              loc='center left', 
              bbox_to_anchor=(1, 0, 0.5, 1))

plt.axis('equal')
plt.tight_layout()

# 保存饼图
plt.savefig('登革热区域分布饼图.png', dpi=300)
print("保存图表：登革热区域分布饼图.png")
plt.close()

C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_16032\1777418163.py:53: UserWarning: Glyph 22320 (\N{CJK UNIFIED IDEOGRAPH-5730}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_16032\1777418163.py:53: UserWarning: Glyph 21306 (\N{CJK UNIFIED IDEOGRAPH-533A}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_16032\1777418163.py:56: UserWarning: Glyph 22320 (\N{CJK UNIFIED IDEOGRAPH-5730}) missing from font(s) Arial.
  plt.savefig('登革热区域分布饼图.png', dpi=300)
C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_16032\1777418163.py:56: UserWarning: Glyph 21306 (\N{CJK UNIFIED IDEOGRAPH-533A}) missing from font(s) Arial.
  plt.savefig('登革热区域分布饼图.png', dpi=300)


保存图表：登革热区域分布饼图.png
